<a href="https://colab.research.google.com/github/chhayankjain/Basic_models/blob/master/Pipeline.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
#Install |
!pip install SpeechRecognition
!pip install youtube_dl

     |████████████████████████████████| 32.8 MB 1.7 MB/s 
     |████████████████████████████████| 1.9 MB 13.4 MB/s 


In [ ]:
from google.colab import drive
drive.mount('/content/drive') 
#changed the code to test version control

Mounted at /content/drive


In [ ]:
#Import the required libraries
import speech_recognition as sr
import scipy.io.wavfile as wav
import pickle as pk
import os
import math
import cv2
from os import listdir
from pathlib import Path

In [ ]:
from __future__ import print_function, division

import torch
import torch.nn as nn
import torch.optim as optim
from torch.optim import lr_scheduler
import numpy as np
import torchvision
from torchvision import datasets, models, transforms
import matplotlib.pyplot as plt
import time
import os
import copy
from PIL import Image
from torch.utils.data import IterableDataset
import queue
import torchvision.transforms.functional as TF
import pickle as pkl

plt.ion()   

In [ ]:
import base64
import io
import time
import cv2
from IPython.display import display
from IPython.display import HTML
from IPython.display import Javascript
import matplotlib
from matplotlib.animation import FuncAnimation
import matplotlib.pyplot as plt
import numpy as np
from PIL import Image
from scipy.signal import medfilt
import tensorflow.compat.v2 as tf
import youtube_dl
from google.colab import drive
from google.colab import output
from google.colab.output import eval_js

In [ ]:
import numpy as np
import argparse
from pathlib import Path
import os
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Dense, Dropout
from tensorflow.keras.applications.mobilenet import MobileNet
from tensorflow.keras.applications.mobilenet import preprocess_input
from tensorflow.keras.preprocessing.image import load_img, img_to_array
import tensorflow as tf

In [ ]:
import pandas as pd

In [ ]:
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")

In [ ]:
#get packages for RepNet
PATH_TO_CKPT = '/tmp/repnet_ckpt/'
!mkdir $PATH_TO_CKPT
!wget -nc -P $PATH_TO_CKPT https://storage.googleapis.com/repnet_ckpt/checkpoint
!wget -nc -P $PATH_TO_CKPT https://storage.googleapis.com/repnet_ckpt/ckpt-88.data-00000-of-00002
!wget -nc -P $PATH_TO_CKPT https://storage.googleapis.com/repnet_ckpt/ckpt-88.data-00001-of-00002
!wget -nc -P $PATH_TO_CKPT https://storage.googleapis.com/repnet_ckpt/ckpt-88.index

# model = get_repnet_model(PATH_TO_CKPT)

--2022-01-21 00:15:04--  https://storage.googleapis.com/repnet_ckpt/checkpoint
Resolving storage.googleapis.com (storage.googleapis.com)... 142.251.5.128, 64.233.184.128, 74.125.133.128, ...
Connecting to storage.googleapis.com (storage.googleapis.com)|142.251.5.128|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 922 [application/octet-stream]
Saving to: ‘/tmp/repnet_ckpt/checkpoint’

checkpoint          100%[===================>]     922  --.-KB/s    in 0s      

2022-01-21 00:15:05 (2.25 MB/s) - ‘/tmp/repnet_ckpt/checkpoint’ saved [922/922]

--2022-01-21 00:15:05--  https://storage.googleapis.com/repnet_ckpt/ckpt-88.data-00000-of-00002
Resolving storage.googleapis.com (storage.googleapis.com)... 74.125.133.128, 74.125.140.128, 173.194.76.128, ...
Connecting to storage.googleapis.com (storage.googleapis.com)|74.125.133.128|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 93435 (91K) [application/octet-stream]
Saving to: ‘/tmp/repnet_c

In [ ]:
#Parameters for RepNet
# Threshold to consider periodicity in entire video.
THRESHOLD = 0.2#@param {type:"number"}

# Threshold to consider periodicity for individual frames in video.
WITHIN_PERIOD_THRESHOLD = 0.5#@param {type:"number"}

# Use this setting for better results when it is 
# known action is repeating at constant speed.
CONSTANT_SPEED = False#@param {type:"boolean"}

# Use median filtering in time to ignore noisy frames.
MEDIAN_FILTER = True#@param {type:"boolean"}

# Use this setting for better results when it is 
# known the entire video is periodic/reapeating and
# has no aperiodic frames.
FULLY_PERIODIC = False#@param {type:"boolean"}

# Visualization video's FPS.
VIZ_FPS = 30#@param {type:"integer"}

In [ ]:
#Class to iterate through the image frames 
class MyDataset(IterableDataset):
    def __init__(self, image_queue):
      self.queue = image_queue

    def read_next_image(self):
        while self.queue.qsize() > 0:
            # you can add transform here
            yield self.queue.get()
        return None

    def __iter__(self):
        return self.read_next_image()

In [ ]:
# class to check synchronization between the audio and the video frames.
# 
class bot_video:
    def __init__(self, directory, video):
        self.dir = directory
        self.video = video
        self.video.replace(' ', '')
        # print("self", self.video)
        self.trained_model = torch.load('/content/drive/My Drive/BOT PIPELINE/trained_model.pt')
        self.class_names = ["back", "front"]
        self.transform = transforms.Compose([transforms.Resize(256), transforms.CenterCrop(224), transforms.ToTensor(), transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])])
        self.image_frames = self.get_image_frames_and_predictions()
        # print(self.image_frames)
        self.audio_frags = self.get_audio_frames()
        self.wordsToCheck = {"camera":"back", "rear camera":"back", "selfie camera":"front", "front camera":"front"}

    def imshow(self, inp, title=None):
        #function to dispay image frames
        """Imshow for Tensor."""
        inp = inp.numpy().transpose((1, 2, 0))
        mean = np.array([0.485, 0.456, 0.406])
        std = np.array([0.229, 0.224, 0.225])
        inp = std * inp + mean
        inp = np.clip(inp, 0, 1)
        plt.imshow(inp)
        if title is not None:
            plt.title(title)
        plt.pause(0.001)  # pause a bit so that plots are updated
    
    def visualize_model_test(self, image_test, num_images=100 ):
        # test the trained model
        was_training = self.trained_model.training
        self.trained_model.eval()
        images_so_far = 0
        fig = plt.figure()

        with torch.no_grad():
            for i, (inputs, labels) in enumerate(image_test):
                inputs = inputs.to(device)
                labels = labels.to(device)

                outputs = self.trained_model(inputs)
                _, preds = torch.max(outputs, 1)

                for j in range(inputs.size()[0]):
                    images_so_far += 1
                    # ax = plt.subplot(num_images//2, 2, images_so_far)
                    # ax.axis('off')
                    plt.title('predicted: {}'.format(class_names[preds[j]]))
                    imshow(inputs.cpu().data[j])

                    if images_so_far == num_images:
                        self.trained_model.train(mode=was_training)
                        return
            self.trained_model.train(mode=was_training)
    
    def get_frame_prediction(self, image_path):
        # image frame prediction (classes : ['back', 'front])
        buffer = queue.Queue()
        new_input = Image.open(image_path)
        new_input = self.transform(new_input)
        buffer.put(new_input)

        dataset = MyDataset(buffer)
        dataloader = torch.utils.data.DataLoader(dataset, batch_size=1)
        for data in dataloader:
            inputs = data.to(device)
            outputs = self.trained_model(inputs) 
            _, preds = torch.max(outputs, 1)
        # print(self.class_names[preds[0]])
        return self.class_names[preds[0]]


    def get_image_frames_and_predictions(self):
        #function to store prediction results of each frame
        images = {}
        path = self.dir+'/video_frames/'+self.video
        # print("video path ", path)
        fi = os.listdir(path)
        for f in fi:
            imagePath = os.path.join(path, f)
            prediction = self.get_frame_prediction(imagePath)
            images[f.split(".")[0]] = prediction
        return images
    
    def get_audio_frames(self):
        # function to retrieve audio frames 
        p = self.dir+'/audio_frames/'+self.video
        # path = self.dir+'/audio frames/'+self.video+'.data'
        path = os.path.join(p, "audioFrag.data")
        # print(os.listdir(self.dir+'/audio_frames'))
        # print(os.listdir(p))
        audioFrags = pkl.load(open(path, "rb"))
        return audioFrags

    def checkSync(self):
        # Function to check the sync between audio and the video frames
        # Possible words checked: rear camera, selfie camera, front facing camera, front camera, camera, display
        notInSync = 0
        frags = {}
        for k in self.audio_frags:
            # check for possible words
            # print(self.audio_frags[k])
            if "rear camera" in self.audio_frags[k]:
                # print("here........", k, self.audio_frags[k], "rear camera")
                preds = self.image_frames[str(k-1)]
                if preds!="back":
                    notInSync= notInSync + 1
                frags[k] = ("rear camera", "back", preds)

            elif "selfie camera" in self.audio_frags[k]:
                # print("here........", k, self.audio_frags[k], "selfie camera")
                preds = self.image_frames[str(k-1)]
                if preds!="front":
                    notInSync= notInSync + 1
                frags[k] = ("selfie camera", "front", preds)
            
            elif "front facing camera" in self.audio_frags[k]:
                # print("here........", k, self.audio_frags[k], "front facing camera")
                preds = self.image_frames[str(k-1)]
                if preds!="front":
                    notInSync= notInSync + 1
                frags[k] = ("front facing camera", "front", preds)
            
            elif "front camera" in self.audio_frags[k]:
                # print("here........", k, self.audio_frags[k], "front camera")
                preds = self.image_frames[str(k-1)]
                if preds!="front":
                    notInSync= notInSync + 1
                frags[k] = ("front camera", "front", preds)

            elif "camera" in self.audio_frags[k]:
                # print("here........", k, self.audio_frags[k], "camera")
                preds = self.image_frames[str(k-1)]
                if preds!="back":
                    notInSync= notInSync + 1
                frags[k] = ("camera", "back", preds)
            
            elif "display" in self.audio_frags[k]:
                # print("here........", k, self.audio_frags[k], "display")
                preds = self.image_frames[str(k-1)]
                if preds!="front":
                    notInSync= notInSync + 1
                frags[k] = ("display", "front", preds)
            
            else:
                continue

        return notInSync, frags


In [ ]:
# Model definition for repNet
layers = tf.keras.layers
regularizers = tf.keras.regularizers

class ResnetPeriodEstimator(tf.keras.models.Model):
  """RepNet model."""

  def __init__(
      self,
      num_frames=64,
      image_size=112,
      base_model_layer_name='conv4_block3_out',
      temperature=13.544,
      dropout_rate=0.25,
      l2_reg_weight=1e-6,
      temporal_conv_channels=512,
      temporal_conv_kernel_size=3,
      temporal_conv_dilation_rate=3,
      conv_channels=32,
      conv_kernel_size=3,
      transformer_layers_config=((512, 4, 512),),
      transformer_dropout_rate=0.0,
      transformer_reorder_ln=True,
      period_fc_channels=(512, 512),
      within_period_fc_channels=(512, 512)):
    super(ResnetPeriodEstimator, self).__init__()

    # Model params.
    self.num_frames = num_frames
    self.image_size = image_size

    self.base_model_layer_name = base_model_layer_name

    self.temperature = temperature

    self.dropout_rate = dropout_rate
    self.l2_reg_weight = l2_reg_weight

    self.temporal_conv_channels = temporal_conv_channels
    self.temporal_conv_kernel_size = temporal_conv_kernel_size
    self.temporal_conv_dilation_rate = temporal_conv_dilation_rate

    self.conv_channels = conv_channels
    self.conv_kernel_size = conv_kernel_size
    # Transformer config in form of (channels, heads, bottleneck channels).
    self.transformer_layers_config = transformer_layers_config
    self.transformer_dropout_rate = transformer_dropout_rate
    self.transformer_reorder_ln = transformer_reorder_ln

    self.period_fc_channels = period_fc_channels
    self.within_period_fc_channels = within_period_fc_channels

    # Base ResNet50 Model.
    base_model = tf.keras.applications.ResNet50V2(
        include_top=False, weights=None, pooling='max')
    self.base_model = tf.keras.models.Model(
        inputs=base_model.input,
        outputs=base_model.get_layer(self.base_model_layer_name).output)

    # 3D Conv on k Frames
    self.temporal_conv_layers = [
        layers.Conv3D(self.temporal_conv_channels,
                      self.temporal_conv_kernel_size,
                      padding='same',
                      dilation_rate=(self.temporal_conv_dilation_rate, 1, 1),
                      kernel_regularizer=regularizers.l2(self.l2_reg_weight),
                      kernel_initializer='he_normal')]
    self.temporal_bn_layers = [layers.BatchNormalization()
                               for _ in self.temporal_conv_layers]

    # Counting Module (Self-sim > Conv > Transformer > Classifier)
    self.conv_3x3_layer = layers.Conv2D(self.conv_channels,
                                        self.conv_kernel_size,
                                        padding='same',
                                        activation=tf.nn.relu)

    channels = self.transformer_layers_config[0][0]
    self.input_projection = layers.Dense(
        channels, kernel_regularizer=regularizers.l2(self.l2_reg_weight),
        activation=None)
    self.input_projection2 = layers.Dense(
        channels, kernel_regularizer=regularizers.l2(self.l2_reg_weight),
        activation=None)

    length = self.num_frames
    self.pos_encoding = tf.compat.v1.get_variable(
        name='resnet_period_estimator/pos_encoding',
        shape=[1, length, 1],
        initializer=tf.compat.v1.truncated_normal_initializer(stddev=0.02))
    self.pos_encoding2 = tf.compat.v1.get_variable(
        name='resnet_period_estimator/pos_encoding2',
        shape=[1, length, 1],
        initializer=tf.compat.v1.truncated_normal_initializer(stddev=0.02))

    self.transformer_layers = []
    for d_model, num_heads, dff in self.transformer_layers_config:
      self.transformer_layers.append(
          TransformerLayer(d_model, num_heads, dff,
                           self.transformer_dropout_rate,
                           self.transformer_reorder_ln))

    self.transformer_layers2 = []
    for d_model, num_heads, dff in self.transformer_layers_config:
      self.transformer_layers2.append(
          TransformerLayer(d_model, num_heads, dff,
                           self.transformer_dropout_rate,
                           self.transformer_reorder_ln))

    # Period Prediction Module.
    self.dropout_layer = layers.Dropout(self.dropout_rate)
    num_preds = self.num_frames//2
    self.fc_layers = []
    for channels in self.period_fc_channels:
      self.fc_layers.append(layers.Dense(
          channels, kernel_regularizer=regularizers.l2(self.l2_reg_weight),
          activation=tf.nn.relu))
    self.fc_layers.append(layers.Dense(
        num_preds, kernel_regularizer=regularizers.l2(self.l2_reg_weight)))

    # Within Period Module
    num_preds = 1
    self.within_period_fc_layers = []
    for channels in self.within_period_fc_channels:
      self.within_period_fc_layers.append(layers.Dense(
          channels, kernel_regularizer=regularizers.l2(self.l2_reg_weight),
          activation=tf.nn.relu))
    self.within_period_fc_layers.append(layers.Dense(
        num_preds, kernel_regularizer=regularizers.l2(self.l2_reg_weight)))

  def call(self, x):
    # Ensures we are always using the right batch_size during train/eval.
    batch_size = tf.shape(x)[0]
    # Conv Feature Extractor.
    x = tf.reshape(x, [-1, self.image_size, self.image_size, 3])
    x = self.base_model(x)
    h = tf.shape(x)[1]
    w = tf.shape(x)[2]
    c = tf.shape(x)[3]
    x = tf.reshape(x, [batch_size, -1, h, w, c])

    # 3D Conv to give temporal context to per-frame embeddings. 
    for bn_layer, conv_layer in zip(self.temporal_bn_layers,
                                    self.temporal_conv_layers):
      x = conv_layer(x)
      x = bn_layer(x)
      x = tf.nn.relu(x)

    x = tf.reduce_max(x, [2, 3])

    # Reshape and prepare embs for output.
    final_embs = x

    # Get self-similarity matrix.
    x = get_sims(x, self.temperature)

    # 3x3 conv layer on self-similarity matrix.
    x = self.conv_3x3_layer(x)
    x = tf.reshape(x, [batch_size, self.num_frames, -1])
    within_period_x = x

    # Period prediction.
    x = self.input_projection(x)
    x += self.pos_encoding
    for transformer_layer in self.transformer_layers:
      x = transformer_layer(x)
    x = flatten_sequential_feats(x, batch_size, self.num_frames)
    for fc_layer in self.fc_layers:
      x = self.dropout_layer(x)
      x = fc_layer(x)

    # Within period prediction.
    within_period_x = self.input_projection2(within_period_x)
    within_period_x += self.pos_encoding2
    for transformer_layer in self.transformer_layers2:
      within_period_x = transformer_layer(within_period_x)
    within_period_x = flatten_sequential_feats(within_period_x,
                                               batch_size,
                                               self.num_frames)
    for fc_layer in self.within_period_fc_layers:
      within_period_x = self.dropout_layer(within_period_x)
      within_period_x = fc_layer(within_period_x)

    return x, within_period_x, final_embs

  @tf.function
  def preprocess(self, imgs):
    # print(imgs)
    # print(self.image_size)
    imgs = tf.cast(imgs, tf.float32)
    imgs -= 127.5
    imgs /= 127.5
    imgs = tf.image.resize(imgs, (self.image_size, self.image_size))
    return imgs


def get_sims(embs, temperature):
  """Calculates self-similarity between batch of sequence of embeddings."""
  batch_size = tf.shape(embs)[0]
  seq_len = tf.shape(embs)[1]
  embs = tf.reshape(embs, [batch_size, seq_len, -1])

  def _get_sims(embs):
    """Calculates self-similarity between sequence of embeddings."""
    dist = pairwise_l2_distance(embs, embs)
    sims = -1.0 * dist
    return sims

  sims = tf.map_fn(_get_sims, embs)
  sims /= temperature
  sims = tf.nn.softmax(sims, axis=-1)
  sims = tf.expand_dims(sims, -1)
  return sims


def flatten_sequential_feats(x, batch_size, seq_len):
  """Flattens sequential features with known batch size and seq_len."""
  x = tf.reshape(x, [batch_size, seq_len, -1])
  return x


# Transformer from https://www.tensorflow.org/tutorials/text/transformer .
def scaled_dot_product_attention(q, k, v, mask):
  """Calculate the attention weights.

  q, k, v must have matching leading dimensions.
  k, v must have matching penultimate dimension, i.e.: seq_len_k = seq_len_v.
  The mask has different shapes depending on its type(padding or look ahead)
  but it must be broadcastable for addition.

  Args:
    q: query shape == (..., seq_len_q, depth)
    k: key shape == (..., seq_len_k, depth)
    v: value shape == (..., seq_len_v, depth_v)
    mask: Float tensor with shape broadcastable
          to (..., seq_len_q, seq_len_k). Defaults to None.

  Returns:
    outputs: shape == (..., seq_len_q, depth_v)
    attention_weights: shape == (..., seq_len_q, seq_len_k)
  """

  matmul_qk = tf.matmul(q, k, transpose_b=True)  # (..., seq_len_q, seq_len_k)

  # scale matmul_qk.
  dk = tf.cast(tf.shape(k)[-1], tf.float32)
  scaled_attention_logits = matmul_qk / tf.math.sqrt(dk)

  # add the mask to the scaled tensor.
  if mask is not None:
    scaled_attention_logits += (mask * -1e9)

  # softmax is normalized on the last axis (seq_len_k) so that the scores
  # add up to 1.
  # (..., seq_len_q, seq_len_k)
  attention_weights = tf.nn.softmax(scaled_attention_logits, axis=-1)

  outputs = tf.matmul(attention_weights, v)  # (..., seq_len_q, depth_v)

  return outputs, attention_weights


def point_wise_feed_forward_network(d_model, dff):
  return tf.keras.Sequential([
      tf.keras.layers.Dense(dff, activation='relu'),
      tf.keras.layers.Dense(d_model)
  ])


class MultiHeadAttention(tf.keras.layers.Layer):
  """Multi-headed attention layer."""

  def __init__(self, d_model, num_heads):
    super(MultiHeadAttention, self).__init__()
    self.num_heads = num_heads
    self.d_model = d_model

    assert d_model % self.num_heads == 0

    self.depth = d_model // self.num_heads

    self.wq = tf.keras.layers.Dense(d_model)
    self.wk = tf.keras.layers.Dense(d_model)
    self.wv = tf.keras.layers.Dense(d_model)

    self.dense = tf.keras.layers.Dense(d_model)

  def split_heads(self, x, batch_size):
    """Split the last dimension into (num_heads, depth)."""
    x = tf.reshape(x, (batch_size, -1, self.num_heads, self.depth))
    return tf.transpose(x, perm=[0, 2, 1, 3])

  def call(self, v, k, q, mask):
    batch_size = tf.shape(q)[0]

    q = self.wq(q)  # (batch_size, seq_len, d_model)
    k = self.wk(k)  # (batch_size, seq_len, d_model)
    v = self.wv(v)  # (batch_size, seq_len, d_model)

    q = self.split_heads(
        q, batch_size)  # (batch_size, num_heads, seq_len_q, depth)
    k = self.split_heads(
        k, batch_size)  # (batch_size, num_heads, seq_len_k, depth)
    v = self.split_heads(
        v, batch_size)  # (batch_size, num_heads, seq_len_v, depth)

    # scaled_attention.shape == (batch_size, num_heads, seq_len_q, depth)
    # attention_weights.shape == (batch_size, num_heads, seq_len_q, seq_len_k)
    scaled_attention, attention_weights = scaled_dot_product_attention(
        q, k, v, mask)

    scaled_attention = tf.transpose(
        scaled_attention,
        perm=[0, 2, 1, 3])  # (batch_size, seq_len_q, num_heads, depth)

    concat_attention = tf.reshape(
        scaled_attention,
        (batch_size, -1, self.d_model))  # (batch_size, seq_len_q, d_model)

    outputs = self.dense(concat_attention)  # (batch_size, seq_len_q, d_model)

    return outputs, attention_weights


class TransformerLayer(tf.keras.layers.Layer):
  """Implements a single transformer layer (https://arxiv.org/abs/1706.03762).
  """

  def __init__(self, d_model, num_heads, dff,
               dropout_rate=0.1,
               reorder_ln=False):
    super(TransformerLayer, self).__init__()

    self.mha = MultiHeadAttention(d_model, num_heads)
    self.ffn = point_wise_feed_forward_network(d_model, dff)

    self.layernorm1 = tf.keras.layers.LayerNormalization(epsilon=1e-6)
    self.layernorm2 = tf.keras.layers.LayerNormalization(epsilon=1e-6)

    self.dropout1 = tf.keras.layers.Dropout(dropout_rate)
    self.dropout2 = tf.keras.layers.Dropout(dropout_rate)

    self.reorder_ln = reorder_ln

  def call(self, x):
    inp_x = x

    if self.reorder_ln:
      x = self.layernorm1(x)

    # (batch_size, input_seq_len, d_model)
    attn_output, _ = self.mha(x, x, x, mask=None)
    attn_output = self.dropout1(attn_output)

    if self.reorder_ln:
      out1 = inp_x + attn_output
      x = out1
    else:
      # (batch_size, input_seq_len, d_model)
      out1 = self.layernorm1(x + attn_output)
      x = out1

    if self.reorder_ln:
      x = self.layernorm2(x)

    # (batch_size, input_seq_len, d_model)
    ffn_output = self.ffn(x)
    ffn_output = self.dropout2(ffn_output)

    if self.reorder_ln:
      out2 = out1 + ffn_output
    else:
      # (batch_size, input_seq_len, d_model)
      out2 = self.layernorm2(out1 + ffn_output)

    return out2


def pairwise_l2_distance(a, b):
  """Computes pairwise distances between all rows of a and all rows of b."""
  norm_a = tf.reduce_sum(tf.square(a), 1)
  norm_a = tf.reshape(norm_a, [-1, 1])
  norm_b = tf.reduce_sum(tf.square(b), 1)
  norm_b = tf.reshape(norm_b, [1, -1])
  dist = tf.maximum(norm_a - 2.0 * tf.matmul(a, b, False, True) + norm_b, 0.0)
  return dist


def get_repnet_model(logdir):
  """Returns a trained RepNet model.

  Args:
    logdir (string): Path to directory where checkpoint will be downloaded.

  Returns:
    model (Keras model): Trained RepNet model.
  """
  # Check if we are in eager mode.
  assert tf.executing_eagerly()

  # Models will be called in eval mode.
  tf.keras.backend.set_learning_phase(0)

  # Define RepNet model.
  model = ResnetPeriodEstimator()
  # tf.function for speed.
  model.call = tf.function(model.call)

  # Define checkpoint and checkpoint manager.
  ckpt = tf.train.Checkpoint(model=model)
  ckpt_manager = tf.train.CheckpointManager(
      ckpt, directory=logdir, max_to_keep=10)
  latest_ckpt = ckpt_manager.latest_checkpoint
  print('Loading from: ', latest_ckpt)
  if not latest_ckpt:
    raise ValueError('Path does not have a checkpoint to load.')
  # Restore weights.
  ckpt.restore(latest_ckpt).expect_partial()

  # Pass dummy frames to build graph.
  model(tf.random.uniform((1, 64, 112, 112, 3)))
  return model


def unnorm(query_frame):
  min_v = query_frame.min()
  max_v = query_frame.max()
  query_frame = (query_frame - min_v) / max(1e-7, (max_v - min_v))
  return query_frame


def create_count_video(frames,
                       per_frame_counts,
                       within_period,
                       score,
                       fps,
                       output_file,
                       delay,
                       plot_count=True,
                       plot_within_period=False,
                       plot_score=False):
  """Creates video with running count and within period predictions.

  Args:
    frames (List): List of images in form of NumPy arrays.
    per_frame_counts (List): List of floats indicating repetition count for
      each frame. This is the rate of repetition for that particular frame.
      Summing this list up gives count over entire video.
    within_period (List): List of floats indicating score between 0 and 1 if the
      frame is inside the periodic/repeating portion of a video or not.
    score (float): Score between 0 and 1 indicating the confidence of the
      RepNet model's count predictions.
    fps (int): Frames per second of the input video. Used to scale the
      repetition rate predictions to Hz.
    output_file (string): Path of the output video.
    delay (integer): Delay between each frame in the output video.
    plot_count (boolean): if True plots the count in the output video.
    plot_within_period (boolean): if True plots the per-frame within period
      scores.
    plot_score (boolean): if True plots the confidence of the model along with
      count ot within_period scores.
  """
  if output_file[-4:] not in ['.mp4', '.gif']:
    raise ValueError('Output format can only be mp4 or gif')
  num_frames = len(frames)

  running_counts = np.cumsum(per_frame_counts)
  final_count = running_counts[-1]
  print(final_count)
  return final_count

def data_uri_to_img(uri):
  """Convert base64image to Numpy array."""
  image = base64.b64decode(uri.split(',')[1], validate=True)
  # Binary string to PIL image.
  image = Image.open(io.BytesIO(image))
  image = image.resize((224, 224))
  # PIL to Numpy array.
  image = np.array(np.array(image, dtype=np.uint8), np.float32)
  return image


def read_video(video_filename, width=224, height=224):
  """Read video from file."""
  print("the name passed here:::::::::::", video_filename)
  cap = cv2.VideoCapture(video_filename)
  fps = cap.get(cv2.CAP_PROP_FPS)
  frames = []
  if cap.isOpened():
    while True:
      success, frame_bgr = cap.read()
      if not success:
        break
      frame_rgb = cv2.cvtColor(frame_bgr, cv2.COLOR_BGR2RGB)
      frame_rgb = cv2.resize(frame_rgb, (width, height))
      frames.append(frame_rgb)
  frames = np.asarray(frames)
#   print("in read video::::::::::::", frames)
  return frames, fps

def get_score(period_score, within_period_score):
  """Combine the period and periodicity scores."""
  within_period_score = tf.nn.sigmoid(within_period_score)[:, 0]
  per_frame_periods = tf.argmax(period_score, axis=-1) + 1
  pred_period_conf = tf.reduce_max(
      tf.nn.softmax(period_score, axis=-1), axis=-1)
  pred_period_conf = tf.where(
      tf.math.less(per_frame_periods, 3), 0.0, pred_period_conf)
  within_period_score *= pred_period_conf
  within_period_score = np.sqrt(within_period_score)
  pred_score = tf.reduce_mean(within_period_score)
  return pred_score, within_period_score


def get_counts(model, frames, strides, batch_size,
               threshold,
               within_period_threshold,
               constant_speed=False,
               median_filter=False,
               fully_periodic=False):
  """Pass frames through model and conver period predictions to count."""
  seq_len = len(frames)
  raw_scores_list = []
  scores = []
  within_period_scores_list = []

  if fully_periodic:
    within_period_threshold = 0.0

  frames = model.preprocess(frames)

  for stride in strides:
    num_batches = int(np.ceil(seq_len/model.num_frames/stride/batch_size))
    raw_scores_per_stride = []
    within_period_score_stride = []
    for batch_idx in range(num_batches):
      idxes = tf.range(batch_idx*batch_size*model.num_frames*stride,
                       (batch_idx+1)*batch_size*model.num_frames*stride,
                       stride)
      idxes = tf.clip_by_value(idxes, 0, seq_len-1)
      curr_frames = tf.gather(frames, idxes)
      curr_frames = tf.reshape(
          curr_frames,
          [batch_size, model.num_frames, model.image_size, model.image_size, 3])

      raw_scores, within_period_scores, _ = model(curr_frames)
      raw_scores_per_stride.append(np.reshape(raw_scores.numpy(),
                                              [-1, model.num_frames//2]))
      within_period_score_stride.append(np.reshape(within_period_scores.numpy(),
                                                   [-1, 1]))
    raw_scores_per_stride = np.concatenate(raw_scores_per_stride, axis=0)
    raw_scores_list.append(raw_scores_per_stride)
    within_period_score_stride = np.concatenate(
        within_period_score_stride, axis=0)
    pred_score, within_period_score_stride = get_score(
        raw_scores_per_stride, within_period_score_stride)
    scores.append(pred_score)
    within_period_scores_list.append(within_period_score_stride)

  # Stride chooser
  argmax_strides = np.argmax(scores)
  chosen_stride = strides[argmax_strides]
  raw_scores = np.repeat(
      raw_scores_list[argmax_strides], chosen_stride, axis=0)[:seq_len]
  within_period = np.repeat(
      within_period_scores_list[argmax_strides], chosen_stride,
      axis=0)[:seq_len]
  within_period_binary = np.asarray(within_period > within_period_threshold)
  if median_filter:
    within_period_binary = medfilt(within_period_binary, 5)

  # Select Periodic frames
  periodic_idxes = np.where(within_period_binary)[0]

  if constant_speed:
    # Count by averaging predictions. Smoother but
    # assumes constant speed.
    scores = tf.reduce_mean(
        tf.nn.softmax(raw_scores[periodic_idxes], axis=-1), axis=0)
    max_period = np.argmax(scores)
    pred_score = scores[max_period]
    pred_period = chosen_stride * (max_period + 1)
    per_frame_counts = (
        np.asarray(seq_len * [1. / pred_period]) *
        np.asarray(within_period_binary))
  else:
    # Count each frame. More noisy but adapts to changes in speed.
    pred_score = tf.reduce_mean(within_period)
    per_frame_periods = tf.argmax(raw_scores, axis=-1) + 1
    per_frame_counts = tf.where(
        tf.math.less(per_frame_periods, 3),
        0.0,
        tf.math.divide(1.0,
                       tf.cast(chosen_stride * per_frame_periods, tf.float32)),
    )
    if median_filter:
      per_frame_counts = medfilt(per_frame_counts, 5)

    per_frame_counts *= np.asarray(within_period_binary)

    pred_period = seq_len/np.sum(per_frame_counts)

  if pred_score < threshold:
    print('No repetitions detected in video as score '
          '%0.2f is less than threshold %0.2f.'%(pred_score, threshold))
    per_frame_counts = np.asarray(len(per_frame_counts) * [0.])

  return (pred_period, pred_score, within_period,
          per_frame_counts, chosen_stride)

In [ ]:
#class definition to perfor neural image assessment 
class nima:
    def __init__(self, directory, out_dir):
        self.out_dir = out_dir
        self.dir = directory
        self.target_size = (224, 224)
        self.imgs = os.listdir(self.dir)
        self.file = os.path.join(self.out_dir, "nima_scores.txt")
    
    def mean_score(self, scores):
        si = np.arange(1, 11, 1)
        mean = np.sum(scores * si)
        return mean

    def std_score(self, scores):
        si = np.arange(1, 11, 1)
        mean = self.mean_score(scores)
        std = np.sqrt(np.sum(((si - mean) ** 2) * scores))
        return std

    def get_scores(self):
        # NIMA mean and std scores are obtained using a pretrained model.
        overall_mean = 0
        overall_std = 0
        with tf.device('/CPU:0'):
            base_model = MobileNet((None, None, 3), alpha=1, include_top=False, pooling='avg', weights=None)
            x = Dropout(0.75)(base_model.output)
            x = Dense(10, activation='softmax')(x)

            model = Model(base_model.input, x)
            model.load_weights('/content/drive/My Drive/BOT PIPELINE/NIMA weights/mobilenet_weights.h5')

            score_list = []            
            tot = len(self.imgs)
            for i in self.imgs:
                img_path = os.path.join(self.dir, i)
                img = load_img(img_path, target_size=self.target_size)
                x = img_to_array(img)
                x = np.expand_dims(x, axis=0)
                x = preprocess_input(x)
                scores = model.predict(x, batch_size=1, verbose=0)[0]
                mean = self.mean_score(scores)
                std = self.std_score(scores)

                file_name = Path(img_path).name.lower()
                score_list.append((file_name, mean))

                print("Evaluating : ", img_path)
                print("NIMA Score : %0.3f +- (%0.3f)" % (mean, std))
                print()

                with open(self.file, "a") as f:
                    f.write("{} : {} +- ({})".format(i, round(mean, 3), round(std, 3)))
                    f.write("\n")

                overall_mean += mean
                overall_std += std

        ret_mean = overall_mean/tot
        ret_std = overall_std/tot
        return ret_mean, ret_std   

In [ ]:
class bot_pipeline:
    def __init__(self, directory, vid_type, channel):
        #Initialize directory names and variables
        self.dir = directory
        self.audio_input_path = os.path.join(self.dir, "audio")
        self.audio_frag_outpath = os.path.join(self.dir, "audio_frames")
        Path(self.audio_frag_outpath).mkdir(parents=True, exist_ok=True)
        #video coming from this ##################################################################
        self.video_input_path = os.path.join(self.dir, "video")
        self.video_frag_outpath = os.path.join(self.dir, "video_frames")
        Path(self.video_frag_outpath).mkdir(parents=True, exist_ok=True)
        self.log_outputs = os.path.join(self.dir, "log_outputs")
        Path(self.log_outputs).mkdir(parents = True, exist_ok = True)

        df_path = os.path.join(self.dir, "bot_features2.csv")
        self.df = pd.read_csv(df_path)
        self.vid_type = vid_type
        self.channel = channel

        #Fill dataframe 
        self.fill_dataframe()

        # #call speech2Text
        self.speech_2_text_main()
        
        #call video frames
        self.getVideoFramesMain()
        
        #call check sync
        self.checkSync()
        
        # Call repnet
        #This has been commented ###################################################################
        self.repNet()

        #check presence of human
        self.detect_human()

        #Get Nima scores
        self.image_assessment()

        #save dataframe
        self.df.to_csv(df_path, index=False)
    
    def fill_dataframe(self):
        #fill the pandas dataframe with the video names and the channel name
        videos = os.listdir(self.video_input_path)
        for v in videos:
            vname = v.split(".")[0]
            self.df = self.df.append([{"Name": vname, "Type":self.vid_type, "Channel":self.channel}], ignore_index=True)
        self.df = self.df.replace(np.nan, 0)

    def speech2TextTrans(self, file_path):
        # Speech to text transcription done using the python library SpeechRecognition
        (source_rate, source_sig) = wav.read(file_path)
        duration_seconds = len(source_sig) / float(source_rate)
        
        review = sr.AudioFile(file_path)
        # print(duration_seconds)
        
        r2 = sr.Recognizer()
        o = 0
        fragResults = {}
        audio2 = []
        
        #audio stride = 10 seconds
        for o in range(0, int(duration_seconds), 10):
            with review as audioFrag:
                r2.adjust_for_ambient_noise(audioFrag)
                audio2.append(r2.record(audioFrag, offset = o, duration = 10))
        counter = 1
        for a in audio2:
            try:
                fragResults[counter] = r2.recognize_google(a)
                counter+=1
            except sr.UnknownValueError:
                # print("Unknown value error")
                continue
        # print(counter) 
        return fragResults
    
    def speech_2_text_main(self):
        #function to convert speech to text. The audio frames are stored in the directory "audio frames"
        directory = os.path.abspath(self.audio_input_path)
        audioFiles = []
        print(self.audio_input_path)
        for file in os.listdir(directory):
            if file.endswith(".wav"):
                audioFiles.append(file)
        print("audio files: ", audioFiles)
                
        for audio in audioFiles:
            path = os.path.join(directory, audio)
            print(path)
            audioFrags = self.speech2TextTrans(path)
            # print(audio, audioFrags)
            fol = os.path.join(self.audio_frag_outpath, audio.split(".")[0])
            Path(fol).mkdir(parents=True, exist_ok=True)
            # save fragments as a pickle file
            pk.dump(audioFrags, open(os.path.join(fol, "audioFrag.data"), "wb"))

        print("done - speech2Text")

    def getFrame(self, filepath, sec, cnt):
        # extract image frames from video using openCV
        # print("here")
        vidcap =cv2.VideoCapture(filepath)
        vidcap.set(cv2.CAP_PROP_POS_MSEC,sec*1000)
        # print(sec, vidcap.get(cv2.CAP_PROP_POS_MSEC))
        hasFrames,image = vidcap.read()
        outpath =  os.path.join(self.video_frag_outpath,(filepath.split(".")[0]).split("/")[-1])
        #save the frames in .jpg format
        if hasFrames:
            cv2.imwrite(outpath+"/"+str(cnt)+".jpg", image)     # save frame as JPG file
            # print("done")
        return hasFrames

    def getVideoFramesMain(self):
        # Main function to extract image frames from video files. The video frames are stored in the directory "video frames"
        folder = self.video_input_path
        files = []
        # ouputPath = []
        for filename in os.listdir(folder):
            filepath = os.path.join(folder, filename)
            files.append(filepath)
            outputPath = os.path.join(self.video_frag_outpath, filename.split(".")[0])
            Path(outputPath).mkdir(parents=True, exist_ok=True)

        for i in range(len(files)):
            sec = 0
            frameRate = 10
            count=0
            success = self.getFrame(files[i], sec, count)
            while success:
                count = count+1
                sec = sec + frameRate
                sec = round(sec, 2)
                success = self.getFrame(files[i], sec, count)
        print("done: video frames")

    def checkSync(self):
        # function to get the outOfSync counts of audio and video frames
        videos = os.listdir(self.video_input_path)
        # syncs = {}
        for v in videos:
            # print(v.split(".")[0])
            vid = v.split(".")[0]
            outpath = os.path.join(self.log_outputs, vid)
            Path(outpath).mkdir(parents=True, exist_ok=True)
            self.file = os.path.join(outpath, "checkSync_outputs.txt")
            b = bot_video(self.dir, vid)
            syncNo, frags = b.checkSync()
            syncs = {"outOfSync":syncNo, "frags":frags}
            self.df.at[self.df['Name'].eq(vid),'Sync'] = syncNo
            f = open(self.file ,"w")
            f.write( str(syncs) )
            f.close()
            print("Sync for "+vid+" ------> done")

            
        print("done sync")

    def repNet(self):
        # Functio  to get the count of repetations in the video files
        videos = os.listdir(self.video_input_path)
        print(videos)
        model = get_repnet_model(PATH_TO_CKPT)
        for v in list(videos):
            PATH_TO_VIDEO_ON_YOUR_DRIVE = os.path.join(self.video_input_path, v)
            print(v)
            imgs, vid_fps = read_video(PATH_TO_VIDEO_ON_YOUR_DRIVE)
            print('Running RepNet...') 
            print(len(imgs), vid_fps)
            # print("in repnet, imgs::::: ", imgs)
            # print("vid_fps:::::::", vid_fps)
            (pred_period, pred_score, within_period, per_frame_counts, chosen_stride) = get_counts( model,
                                                                                                    imgs,
                                                                                                    strides=[1,2,3,4],
                                                                                                    batch_size=20,
                                                                                                    threshold=THRESHOLD,
                                                                                                    within_period_threshold=WITHIN_PERIOD_THRESHOLD,
                                                                                                    constant_speed=CONSTANT_SPEED,
                                                                                                    median_filter=MEDIAN_FILTER,
                                                                                                    fully_periodic=FULLY_PERIODIC)
            print("got rep counts")
            c = create_count_video(imgs,
                                   per_frame_counts,
                                    within_period,
                                    score=pred_score,
                                    fps=vid_fps,
                                    output_file='/tmp/debug_video.mp4',
                                    delay=1000/VIZ_FPS,
                                    plot_count=True,
                                    plot_within_period=True,
                                    plot_score=True)
            print(v, round(c))
            vid = v.split(".")[0]
            self.df.at[self.df['Name'].eq(vid),'Repetation'] = round(c)
        print("repNet done")
   
    def load_human_detection_model(self):
        # pretrained models to detect human/person in video frames
        proto_path = os.path.join(self.dir, "Human detection/MobileNetSSD_deploy.prototxt.txt")
        model_path = os.path.join(self.dir, "Human detection/MobileNetSSD_deploy.caffemodel")
        return cv2.dnn.readNetFromCaffe(proto_path, model_path)

    def detect_person(self, net, image):
        # function to detect human in image files
        CLASSES = ["background", "aeroplane", "bicycle", "bird", "boat",
               "bottle", "bus", "car", "cat", "chair", "cow", "diningtable",
               "dog", "horse", "motorbike", "person", "pottedplant", "sheep",
               "sofa", "train", "tvmonitor"]

        # load the input image and construct an input blob for the image
        # by resizing to a fixed 300x300 pixels and then normalizing it
        # (note: normalization is done via the authors of the MobileNet SSD
        # implementation)
        (h, w) = image.shape[:2]
        blob = cv2.dnn.blobFromImage(cv2.resize(image, (300, 300)), 0.007843, (300, 300), 127.5)

        # pass the blob through the network and obtain the detections and
        # predictions
        net.setInput(blob)
        detections = net.forward()

        # loop over the detections
        for i in np.arange(0, detections.shape[2]):
            # extract the confidence (i.e., probability) associated with the
            # prediction
            confidence = detections[0, 0, i, 2]

            # filter out weak detections by ensuring the `confidence` is
            # greater than the minimum confidence
            if confidence > 0.2:
                # extract the index of the class label from the `detections`,
                # then compute the (x, y)-coordinates of the bounding box for
                # the object
                idx = int(detections[0, 0, i, 1])
                box = detections[0, 0, i, 3:7] * np.array([w, h, w, h])
                (startX, startY, endX, endY) = box.astype("int")

                # display the prediction
                label = "{}: {:.2f}%".format(CLASSES[idx], confidence * 100)
                if 'person' in label:
                    print("[INFO] {}".format(label))
                    return True
        return False

    def detect_human(self):
        # main function for human detection. Save the predicted results in the dataframe
        videos = os.listdir(self.video_input_path)
        for v in videos:
            # result = 0
            p = os.path.join(self.video_frag_outpath, v.split(".")[0])
            imgs = os.listdir(p)
            print(v)
            cnt = 0
            for i in imgs:
                img_path = os.path.join(p, i)
                image = cv2.imread(img_path)
                net = self.load_human_detection_model()
                if self.detect_person(net, image):
                    # print("Found person in  ", i)
                    # result = 1
                    cnt+=1
            
            # if result==0:
            #     print("Person not found in ", v)

            print("Human in "+ v+" -------------------->" + str(cnt))
            vid = v.split(".")[0]
            self.df.at[self.df['Name'].eq(vid),'Human detected'] = cnt
        print("human detection done")
    
    def image_assessment(self):
        # function to get NIMA scores
        videos = os.listdir(self.video_input_path)
        for v in videos:
            outputPath = os.path.join(self.log_outputs, v.split(".")[0])
            Path(outputPath).mkdir(parents=True, exist_ok=True)
            print("NIMA SCORES for ", v)
            p = os.path.join(self.video_frag_outpath, v.split(".")[0])
            n = nima(p, outputPath)
            mean, std = n.get_scores()
            vid = v.split(".")[0]
            self.df.at[self.df['Name'].eq(vid),'Nima Mean'] = mean
            self.df.at[self.df['Name'].eq(vid),'Nima STD'] = std
            print("NIMA done for ", v)
        print("NIMA done")    


In [ ]:
# Main
import pandas as pd
d = "/content/drive/My Drive/BOT PIPELINE"

# type of video: human/bot
vid_type = "bot"


# video channel name 
channel = "Tech Key"

#create class variable
b = bot_pipeline(d, vid_type, channel)

/content/drive/My Drive/BOT PIPELINE/audio
audio files:  ['Oppo F17 Pro - 100_ Bezel-Less, Ultra Thin,  Android 10.0 _ Price _ Release Date !.wav', 'POCO X2 - First Look _ Final Price, Release Date _ Full Specification !.wav', 'Samsung A100 - 7.2 Inch Display, 5 Back Camera, 7000 mAh Battery _ Price _ Launch Date !.wav', 'Xiaomi Mi Mix Alpha 5G - First Look _ 7.9 Inch Curved Screen, 108MP Camera _ Price _ Release Date !.wav', 'Oppo F15 Pro - 100_ Bezel-Less, 86MP Camera,  Android 10.0, 5G _  Price _ Release Date !.wav', 'Xiaomi Mi Mix 4 - 108MP Camera, Ultra HD+ Display, 5G Connectivity, Price _ Launch Date !.wav', 'Samsung M50 - Ultra Thin, 40MP Selfie Camera, 5G Connectivity _ Price _ Release Date !.wav', 'Vivo V17 Pro - 7.0 Inch Display,  6 Camera, Ultra HD _ Price _ Launch Date !.wav', 'Realme 5 - 5G Connectivity, 100_ Bezel-Less, 256GB Storage _ Price _ Release Date !.wav', 'Redmi Note 9 - 102MP Camera, 4K Display, 5000mAh Battery, 5G _ Price _ Launch Date !.wav', 'Realme 5 Pro - 

/usr/local/lib/python3.7/dist-packages/keras/backend.py:414: UserWarning: `tf.keras.backend.set_learning_phase` is deprecated and will be removed after 2020-10-11. To update it, simply pass a True/False value to the `training` argument of the `__call__` method of your layer or model.
  warnings.warn('`tf.keras.backend.set_learning_phase` is deprecated and '


Loading from:  /tmp/repnet_ckpt/ckpt-88
Oppo F17 Pro - 100_ Bezel-Less, Ultra Thin,  Android 10.0 _ Price _ Release Date !.webm
the name passed here::::::::::: /content/drive/My Drive/BOT PIPELINE/video/Oppo F17 Pro - 100_ Bezel-Less, Ultra Thin,  Android 10.0 _ Price _ Release Date !.webm
Running RepNet...
5459 29.97002997002997
got rep counts
7.458946090191603
Oppo F17 Pro - 100_ Bezel-Less, Ultra Thin,  Android 10.0 _ Price _ Release Date !.webm 7
Samsung A91 - 108MP Camera, 5G Connectivity, Ultra HD Display _ Price _ Launch Date !.mp4
the name passed here::::::::::: /content/drive/My Drive/BOT PIPELINE/video/Samsung A91 - 108MP Camera, 5G Connectivity, Ultra HD Display _ Price _ Launch Date !.mp4
Running RepNet...
5708 29.97017683874491
got rep counts
9.912282805889845
Samsung A91 - 108MP Camera, 5G Connectivity, Ultra HD Display _ Price _ Launch Date !.mp4 10
POCO X2 - First Look _ Final Price, Release Date _ Full Specification !.mp4
the name passed here::::::::::: /content/drive/